In [2]:
from datetime import date, datetime, timedelta

import pandas as pd

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", "{:,.2f}".format)

In [19]:
fiscal_start_month = 3
fiscal_anchor_offset = "FEB"

In [99]:
df = pd.DataFrame(
    {"date": pd.date_range(start="2024-12-01", end="2025-03-31", freq="D")}
)


def _add_standard_columns(df):
    dt = df["date"].dt
    iso = dt.isocalendar()
    period_w = dt.to_period("W")
    period_m = dt.to_period("M")
    period_q = dt.to_period("Q")
    period_y = dt.to_period("Y")

    current_date = pd.Timestamp.today()
    current_period_w = current_date.to_period("W")
    current_period_m = current_date.to_period("M")
    current_period_q = current_date.to_period("Q")
    current_period_y = current_date.to_period("Y")

    return df.assign(
        year=dt.year,
        month=dt.month,
        month_abbr=dt.strftime("%b"),
        year_month=dt.strftime("%b %Y"),
        year_month_number=dt.year * 12 + dt.month - 1,
        quarter=dt.quarter,
        year_quarter=dt.to_period("Q"),
        iso_week=iso.week,
        day_of_week=iso.day,
        day_name=dt.strftime("%a"),
        start_of_year=period_y.dt.start_time,
        end_of_year=period_y.dt.end_time.dt.floor("D"),
        start_of_quarter=period_q.dt.start_time,
        end_of_quarter=period_q.dt.end_time.dt.floor("D"),
        start_of_month=period_m.dt.start_time,
        end_of_month=period_m.dt.end_time.dt.floor("D"),
        start_of_week=period_w.dt.start_time,
        end_of_week=period_w.dt.end_time.dt.floor("D"),
        year_offset=dt.year - current_period_y.year,
        quarter_offset=(
            ((dt.year - current_date.year) * 4)
            + (dt.quarter - current_date.quarter)
        ),
        month_offset=(
            ((dt.year - current_date.year) * 12)
            + (dt.month - current_date.month)
        ),
        week_offset=(
            period_w.dt.start_time - current_period_w.start_time
        ).dt.days
        // 7,
    )


def _add_fiscal_columns(df):
    dt = df["date"].dt

    period_y = dt.to_period("Y-FEB")
    period_q = dt.to_period("Q-FEB")
    fiscal_start_month = period_y.dt.start_time.dt.month

    current_date = pd.Timestamp.today()
    current_period_y = current_date.to_period("Y-FEB")
    current_period_q = current_date.to_period("Q-FEB")

    return df.assign(
        fiscal_year=period_y.dt.year,
        fiscal_month=((dt.month - fiscal_start_month) % 12) + 1,
        fiscal_quarter=period_q.dt.quarter,
        fiscal_year_quarter=period_q,
        start_of_fiscal_year=period_y.dt.start_time,
        end_of_fiscal_year=period_y.dt.end_time.dt.normalize(),
        start_of_fiscal_quarter=period_q.dt.start_time,
        end_of_fiscal_quarter=period_q.dt.end_time.dt.floor("D"),
        fiscal_year_offset=period_y.dt.year - current_period_y.year,
        fiscal_quarter_offset=(
            ((period_y.dt.year - current_period_y.year) * 4)
            + (period_q.dt.quarter - current_period_q.quarter)
        ),
    )


df = df.pipe(_add_standard_columns).pipe(_add_fiscal_columns)
display(df)

,date,year,month,month_abbr,year_month,year_month_number,quarter,year_quarter,iso_week,day_of_week,day_name,start_of_year,end_of_year,start_of_quarter,end_of_quarter,start_of_month,end_of_month,start_of_week,end_of_week,year_offset,quarter_offset,month_offset,week_offset,fiscal_year,fiscal_month,fiscal_quarter,fiscal_year_quarter,start_of_fiscal_year,end_of_fiscal_year,start_of_fiscal_quarter,end_of_fiscal_quarter,fiscal_year_offset,fiscal_quarter_offset
0,2024-12-01,2024,12,Dec,Dec 2024,24299,4,2024Q4,48,7,Sun,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-11-25,2024-12-01,-1,-1,-2,-13,2025,10,4,2025Q4,2024-03-01,2025-02-28,2024-12-01,2025-02-28,0,0
1,2024-12-02,2024,12,Dec,Dec 2024,24299,4,2024Q4,49,1,Mon,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-02,2024-12-08,-1,-1,-2,-12,2025,10,4,2025Q4,2024-03-01,2025-02-28,2024-12-01,2025-02-28,0,0
2,2024-12-03,2024,12,Dec,Dec 2024,24299,4,2024Q4,49,2,Tue,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-02,2024-12-08,-1,-1,-2,-12,2025,10,4,2025Q4,2024-03-01,2025-02-28,2024-12-01,2025-02-28,0,0
3,2024-12-04,2024,12,Dec,Dec 2024,24299,4,2024Q4,49,3,Wed,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-02,2024-12-08,-1,-1,-2,-12,2025,10,4,2025Q4,2024-03-01,2025-02-28,2024-12-01,2025-02-28,0,0
4,2024-12-05,2024,12,Dec,Dec 2024,24299,4,2024Q4,49,4,Thu,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-02,2024-12-08,-1,-1,-2,-12,2025,10,4,2025Q4,2024-03-01,2025-02-28,2024-12-01,2025-02-28,0,0
5,2024-12-06,2024,12,Dec,Dec 2024,24299,4,2024Q4,49,5,Fri,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-02,2024-12-08,-1,-1,-2,-12,2025,10,4,2025Q4,2024-03-01,2025-02-28,2024-12-01,2025-02-28,0,0
6,2024-12-07,2024,12,Dec,Dec 2024,24299,4,2024Q4,49,6,Sat,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-02,2024-12-08,-1,-1,-2,-12,2025,10,4,2025Q4,2024-03-01,2025-02-28,2024-12-01,2025-02-28,0,0
7,2024-12-08,2024,12,Dec,Dec 2024,24299,4,2024Q4,49,7,Sun,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-02,2024-12-08,-1,-1,-2,-12,2025,10,4,2025Q4,2024-03-01,2025-02-28,2024-12-01,2025-02-28,0,0
8,2024-12-09,2024,12,Dec,Dec 2024,24299,4,2024Q4,50,1,Mon,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-09,2024-12-15,-1,-1,-2,-11,2025,10,4,2025Q4,2024-03-01,2025-02-28,2024-12-01,2025-02-28,0,0
9,2024-12-10,2024,12,Dec,Dec 2024,24299,4,2024Q4,50,2,Tue,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-09,2024-12-15,-1,-1,-2,-11,2025,10,4,2025Q4,2024-03-01,2025-02-28,2024-12-01,2025-02-28,0,0


In [104]:
import pandas as pd


class DateProcessor:
    def __init__(self, df):
        self.df = df.copy()
        self.dt = self.df["date"].dt
        self.current_date = pd.Timestamp.today()

        self.current_std = {
            "year": self.current_date.to_period("Y"),
            "quarter": self.current_date.to_period("Q"),
            "month": self.current_date.to_period("M"),
            "week": self.current_date.to_period("W"),
        }

        self.current_fisc = {
            "year": self.current_date.to_period("Y-FEB"),
            "quarter": self.current_date.to_period("Q-FEB"),
        }

    def _compute_period_details(self, period_series):
        """Centralized handling of period start/end dates"""
        return (
            period_series.dt.start_time,
            period_series.dt.end_time.dt.floor("D"),  # Flooring happens HERE
        )

    def add_standard_columns(self):
        """Handle standard calendar columns"""
        iso = self.dt.isocalendar()

        # Create all period series first
        periods = {
            "year": self.dt.to_period("Y"),
            "quarter": self.dt.to_period("Q"),
            "month": self.dt.to_period("M"),
            "week": self.dt.to_period("W"),
        }

        # Compute start/end dates using helper
        period_details = {
            k: self._compute_period_details(v) for k, v in periods.items()
        }

        # Calculate offsets using original logic
        offsets = {
            "year": periods["year"].dt.year - self.current_std["year"].year,
            "quarter": (
                (
                    periods["quarter"].dt.year
                    - self.current_std["quarter"].dt.year
                )
                * 4
                + (
                    periods["quarter"].dt.quarter
                    - self.current_std["quarter"].dt.quarter
                )
            ),
            "month": (
                (periods["month"].dt.year - self.current_std["month"].dt.year)
                * 12
                + (
                    periods["month"].dt.month
                    - self.current_std["month"].dt.month
                )
            ),
            "week": (
                (
                    periods["week"].dt.start_time
                    - self.current_std["week"].dt.start_time
                ).dt.days
                // 7
            ),
        }

        # Assign all columns
        self.df = self.df.assign(
            year=self.dt.year,
            month=self.dt.month,
            month_abbr=self.dt.strftime("%b"),
            year_month=self.dt.strftime("%b %Y"),
            year_month_number=self.dt.year * 12 + self.dt.month - 1,
            quarter=self.dt.quarter,
            year_quarter=periods["quarter"],
            iso_week=iso.week,
            day_of_week=iso.day,
            day_name=self.dt.strftime("%a"),
            # Use precomputed period details
            start_of_year=period_details["year"][0],
            end_of_year=period_details["year"][1],
            start_of_quarter=period_details["quarter"][0],
            end_of_quarter=period_details["quarter"][1],
            start_of_month=period_details["month"][0],
            end_of_month=period_details["month"][1],
            start_of_week=period_details["week"][0],
            end_of_week=period_details["week"][1],
            # Offsets
            year_offset=offsets["year"],
            quarter_offset=offsets["quarter"],
            month_offset=offsets["month"],
            week_offset=offsets["week"],
        )
        return self

    def add_fiscal_columns(self):
        """Handle fiscal calendar columns"""
        periods = {
            "year": self.dt.to_period("Y-FEB"),
            "quarter": self.dt.to_period("Q-FEB"),
        }

        period_details = {
            k: self._compute_period_details(v) for k, v in periods.items()
        }

        # Fiscal month calculation
        fiscal_start_month = periods["year"].dt.start_time.dt.month
        fiscal_month = ((self.dt.month - fiscal_start_month) % 12) + 1

        # Fiscal offsets
        fiscal_offsets = {
            "year": periods["year"].dt.year - self.current_fisc["year"].year,
            "quarter": (
                (periods["year"].dt.year - self.current_fisc["year"].dt.year)
                * 4
                + (
                    periods["quarter"].dt.quarter
                    - self.current_fisc["quarter"].dt.quarter
                )
            ),
        }

        self.df = self.df.assign(
            fiscal_year=periods["year"].dt.year,
            fiscal_month=fiscal_month,
            fiscal_quarter=periods["quarter"].dt.quarter,
            fiscal_year_quarter=periods["quarter"],
            start_of_fiscal_year=period_details["year"][0],
            end_of_fiscal_year=period_details["year"][1],
            start_of_fiscal_quarter=period_details["quarter"][0],
            end_of_fiscal_quarter=period_details["quarter"][1],
            fiscal_year_offset=fiscal_offsets["year"],
            fiscal_quarter_offset=fiscal_offsets["quarter"],
        )
        return self

    def get_processed_df(self):
        return self.df

In [79]:
df.dtypes

date                       datetime64[ns]
fiscal_year                         int64
fiscal_month                        int32
fiscal_quarter                      int64
fiscal_year_quarter         period[Q-FEB]
start_of_fiscal_year       datetime64[ns]
end_of_fiscal_year         datetime64[ns]
start_of_fiscal_quarter    datetime64[ns]
end_of_fiscal_quarter      datetime64[ns]
fiscal_year_offset                  int64
fiscal_quarter_offset               int64
current_period_qs          datetime64[ns]
current_period_qe          datetime64[ns]
dtype: object

In [ ]:
def _create_period_boundaries(self, periods, period_type):
    """Generate start/end time columns for different periods"""
    prefix = "fiscal_" if period_type == "fiscal" else ""
    boundaries = {}

    # List of periods to process based on period_type
    periods_to_process = (
        ["year", "quarter"]
        if period_type == "fiscal"
        else ["year", "quarter", "month", "week"]
    )

    for period in periods_to_process:
        p = periods[f"period_{period}"]
        boundaries.update(
            {
                f"start_of_{prefix}{period}": p.dt.start_time,
                f"end_of_{prefix}{period}": p.dt.end_time.dt.floor("D"),
            }
        )

    return boundaries